In [1]:
import cv2
# Importing the opencv library
import imutils
from imutils import paths
# Importing the library that supports basic image processing functions
import numpy as np
# Importing the array operations library for python
import os
# Importing the library which supports standard systems commands
from scipy.cluster.vq import *
# Importing the library which classifies set of observations into clusters
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics as metrics

In [2]:
imagePaths = list(imutils.paths.list_images("images"))

labels = []
descriptorList = []

sift = cv2.xfeatures2d.SIFT_create()
# descriptorExtractor = cv2.DescriptorExtractor_create("SIFT")

for (i, imagePath) in enumerate(imagePaths):
    image = cv2.imread(imagePath)
#     gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    label = imagePath.split(os.path.sep)[-1].split("_")[1][0]
#     keyPoints = featureDetector.detect(gray_image)
#     keyPoints, descriptor = descriptorExtractor.compute(image, keyPoints)
    keyPoints, descriptor = sift.detectAndCompute(image, None)
    descriptorList.append((imagePath, descriptor))
    labels.append(label)
    
descriptors = descriptorList[0][1]
    
for image_path, descriptor in descriptorList[1:]:
    descriptors = np.vstack((descriptors, descriptor))
    

In [3]:
k = 500

voc,  variance = kmeans(descriptors, k, 1)
print (variance)
print (len(voc))

256.99744
500


In [4]:
imageFeatures = np.zeros((len(imagePaths), k), "float32")
for i in range(len(imagePaths)):
    words, distance = vq(descriptorList[i][1],voc)
    for w in words:
        imageFeatures[i][w] += 1

In [5]:
numberOccurences = np.sum( (imageFeatures > 0) * 1, axis = 0)
# print (len(numberOccurences))
idf = np.array(np.log((1.0*len(imagePaths)+1) / (1.0*numberOccurences + 1)), 'float32')
print (len(idf))

500


In [6]:
stdSlr = StandardScaler().fit(imageFeatures)
imageFeatures = stdSlr.transform(imageFeatures)

In [7]:
# X_train, X_test, y_train, y_test = train_test_split(imageFeatures, labels, test_size=0.4, random_state=4)

# # clf = cv2.ml.KNearest_create()
# # clf.train(imageFeatures, labels)
# knn = KNeighborsClassifier(n_neighbors=5)
# knn.fit(X_train, y_train)
# y_pred = knn.predict(X_test)
# print(metrics.accuracy_score(y_test, y_pred))

0.5


In [8]:
clf = cv2.ml.KNearest_create()
clf.train(imageFeatures, cv2.ml.ROW_SAMPLE, np.asarray(labels, dtype=np.float32))

True

In [20]:
imagePaths_test = list(imutils.paths.list_images("test"))

print (len(imagePaths_test))

labels_test = []
descriptorList_test = []

sift_test = cv2.xfeatures2d.SIFT_create()
# descriptorExtractor = cv2.DescriptorExtractor_create("SIFT")

for (i, imagePath) in enumerate(imagePaths_test):
    image = cv2.imread(imagePath)
#     gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    label = imagePath.split(os.path.sep)[-1].split("_")[1][0]
#     keyPoints = featureDetector.detect(gray_image)
#     keyPoints, descriptor = descriptorExtractor.compute(image, keyPoints)
    keyPoints, descriptor = sift.detectAndCompute(image, None)
    descriptorList_test.append((imagePath, descriptor))
    labels_test.append(label)
    
descriptors_test = descriptorList[0][1]
    
for image_path, descriptor in descriptorList_test[1:]:
    descriptors_test = np.vstack((descriptors, descriptor))
    
imageFeatures_test = np.zeros((len(imagePaths_test), k), "float32")
for i in range(len(imagePaths_test)):
    words, distance = vq(descriptorList_test[i][1],voc)
    for w in words:
        imageFeatures_test[i][w] += 1
        
# print (len(imageFeatures_test))
        
nbr_occurences_test = np.sum( (imageFeatures_test > 0) * 1, axis = 0)  # Getting the number of occurrences of each word
idf_test = np.array(np.log((1.0*len(imagePaths_test)+1) / (1.0*nbr_occurences_test + 1)), 'float32')

9


In [26]:
ret, results, neighbours ,dist = clf.findNearest(imageFeatures_test, k=15)

In [27]:
# print( "result:  {}\n".format(results) )
print (results)
print (labels_test)

# print( "neighbours:  {}\n".format(neighbours) )
# print( "distance:  {}\n".format(dist) )

[[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
['0', '1', '1', '0', '0', '1', '0', '1', '0']
